# Principal Component Analysis

BitTiger DS501

### Load data

In [ ]:
import pandas as pd
import numpy as np

df_hitters = pd.read_csv('../data/hitters.csv')

# Dropping NAs
df_hitters.dropna(inplace=True)

In [ ]:
df_hitters.head()

In [ ]:
df_hitters['Division'].value_counts()

### Cleaning the data

In [ ]:
# Binarizing columns
def map_binary(df, col):
    vals = df[col].unique()
    df[col] = df[col].apply(lambda x: 0 if x == vals[0] else 1)

map_binary(df_hitters, 'League')
map_binary(df_hitters, 'NewLeague')
map_binary(df_hitters, 'Division')

### Get features and target

In [ ]:
feature_names = df_hitters.columns.difference(['Salary'])

X = df_hitters[feature_names].astype(float).values

y = df_hitters['Salary'].values

### Stardardize features

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Use PCA to transform data and get princial components

In [ ]:
from sklearn.decomposition import PCA

n_col = X_train.shape[1]

pca = PCA(n_components=n_col)

train_components = pca.fit_transform(X_train)# fit: get V, lambda; transform: X_train*V
test_components = pca.transform(X_test)


In [ ]:
# transformed data in new space has the same dimension as original data
print(X_test.shape)
print(test_components.shape)

In [ ]:
# transformed by sklearn
train_components

In [ ]:
# transformed by M*V in class, they are equivalent!
train_components_2 = X_train.dot(pca.components_.T)
train_components_2

In [ ]:
print(pca.explained_variance_.shape)
print(pca.explained_variance_)

In [ ]:
# Inspect the principal axes in feature space
print(pca.components_.shape)
print(pca.components_)

In [ ]:
pca.components_.T.dot(pca.components_)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

In [ ]:
plt.imshow(np.cov(pca.components_.T))
plt.show()

In [ ]:
# visualize the covariance of transformed feature matrix
plt.imshow(np.cov(train_components.T))
plt.show()

### Let's do it by solving eigenvalue problem and compare

In [ ]:
import copy
from numpy.linalg import eigh

# fit
cov_train = np.cov(X_train.T)
lambdas, v = eigh(cov_train)

# eigh order lambda from small to large, so flip
lambdas = lambdas[::-1]
v = -np.flip(v,axis=1)

print(lambdas.shape)
print(v.shape)

In [ ]:
# compare variances
print(pca.explained_variance_)
print(lambdas)

In [ ]:
# compare components
v.T - pca.components_

In [ ]:
# compare tranformed training data
# eigen value sovler:
train_components_eig = X_train.dot(v)
train_components_eig

In [ ]:
# sklearn solver:
train_components

# they are equivalent!

### Let's continue with sklearn pca sovler

### See how much variance the principal components explain

In [ ]:
pca_range = np.arange(n_col) + 1

pca_names = ['PCA_%s' % i for i in pca_range]

plt.bar(pca_range, pca.explained_variance_, align='center')

xticks = plt.xticks(pca_range, pca_names, rotation=90)

plt.ylabel('Variance Explained')

### See how much (percentage of) variance the principal components explain

In [ ]:
pca_range = np.arange(n_col) + 1

pca_names = ['PCA_%s' % i for i in pca_range]

plt.bar(pca_range, pca.explained_variance_ratio_, align='center')

xticks = plt.xticks(pca_range, pca_names, rotation=90)

plt.ylabel('Proportion of Variance Explained')

### How to determine k: percent of variance explained

In [ ]:
pca_range = np.arange(n_col) + 1

pca_names = ['PCA_%s' % i for i in pca_range]

plt.bar(pca_range, pca.explained_variance_ratio_, align='center')
plt.plot(pca_range, np.cumsum(pca.explained_variance_ratio_), 'g-')
plt.plot(pca_range, 0.9*np.ones(len(pca_range)), 'r-')

xticks = plt.xticks(pca_range, pca_names, rotation=90)

plt.ylabel('Proportion of Variance Explained')

### Get train and test error with K-Fold cross validation.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

train_mse_arr = np.array([])
test_mse_arr = np.array([])

for i in pca_range:

    train_subset = train_components[:, :i]

    pca_linear = LinearRegression()

    pca_linear.fit(train_subset, y_train)

    # Get train error
    train_mse = cross_val_score(pca_linear, train_subset, y=y_train,
                                scoring='neg_mean_squared_error', cv=10) * -1
    train_mse_arr = np.append(train_mse_arr, train_mse.mean())
    
    # Get test error
    test_set = test_components[:, :i]
    test_mse = mean_squared_error(pca_linear.predict(test_set), y_test)
    test_mse_arr = np.append(test_mse_arr, test_mse)

### Plot train mse

In [ ]:
plt.plot(pca_range, train_mse_arr, marker='o', color='b', alpha=.5, label='train mse')
plt.ylabel('MSE', fontsize=14)
plt.xlabel('Principal Components Included in Model', fontsize=14)
plt.legend(loc='best')

best_train_mse_pca, min_train_mse = np.argmin(train_mse_arr) + 1, np.min(train_mse_arr)

plt.axvline(best_train_mse_pca, color='b', ls='--', alpha=.3)

print('# of PCs that gives lowest train MSE:', best_train_mse_pca, '@', min_train_mse)

### Plot test mse

In [ ]:
plt.plot(pca_range, test_mse_arr, marker='o', color='b', alpha=.5, label='test mse')
plt.ylabel('MSE', fontsize=14)
plt.xlabel('Principal Components Included in Model', fontsize=14)
plt.legend(loc='best')

best_test_mse_pca, min_test_mse = np.argmin(test_mse_arr) + 1, np.min(test_mse_arr)

plt.axvline(best_test_mse_pca, color='b', ls='--', alpha=.3)

print('# of PCs that gives lowest test MSE:', best_test_mse_pca, '@', min_test_mse)

### Plot train and test mse

In [ ]:
plt.plot(pca_range, train_mse_arr, marker='o', color='b', alpha=.5, label='train mse')
plt.plot(pca_range, test_mse_arr, marker='o', color='r', alpha=.5, label='test mse')
plt.ylabel('MSE', fontsize=14)
plt.xlabel('Principal Components Included in Model', fontsize=14)
plt.legend(loc='best')